In [26]:
using Downloads
using DataFrames
using CSV
using Dates
using TimeZones
using Chain
import Arrow

┌ Info: Precompiling Arrow [69666777-d1a9-59fb-9406-91d4454c9d45]
└ @ Base loading.jl:1317


In [27]:
include("../src/utils.jl")
include("../src/feed/arrow.jl")

ENV["COLUMNS"] = 300
ENV["LINES"] = 10

10

In [29]:
ts2dt(123213213)

1970-01-02T10:13:33.213

In [30]:
s = "Use match(pat, str) to get the matched substring from a string."
p = r"\b(\w[A-Za-z0-9_]+)\(([^\)]*)\)"
m = match(p, s)
# => RegexMatch("match(pat, str)", 1="match", 2="pat, str")
# m.match #=> "match(pat, str)"
m.match

"match(pat, str)"

In [31]:
# out_path = "./data/csv/monthly/ETHUSDT-1m"
out_path = "C:\\Dev\\julia_dev\\fxrobot\\data\\csv\\monthly"
# out_path = "./data/csv/monthly"
println(out_path)
d= readdir(abspath(out_path),join=true);
# d= readdir(out_path,join=true);
d[isdir.(d)]
d[.!isdir.(d)]
d

C:\Dev\julia_dev\fxrobot\data\csv\monthly


11-element Vector{String}:
 "C:\\Dev\\julia_dev\\fxrobot\\data\\csv\\monthly\\ETHUSDT-1m-2021-01.csv"
 "C:\\Dev\\julia_dev\\fxrobot\\data\\csv\\monthly\\ETHUSDT-1m-2021-02.csv"
 "C:\\Dev\\julia_dev\\fxrobot\\data\\csv\\monthly\\ETHUSDT-1m-2021-03.csv"
 ⋮
 "C:\\Dev\\julia_dev\\fxrobot\\data\\csv\\monthly\\ETHUSDT-1m-2021-10.csv"
 "C:\\Dev\\julia_dev\\fxrobot\\data\\csv\\monthly\\ETHUSDT-1m-2021-11.csv"

In [36]:
out_path = abspath("../data/csv/monthly/")
d = readdir(out_path,join=true)
lst_files = d[isfile.(d)]
regexp = r"(.*)(ETHUSDT-1m)(.*)(\.csv)"
# regexp = r"(.*)"
df = DataFrame()
for f in lst_files
    m = match(regexp,f)
    println(m.match)
    d = DataFrame(CSV.File(m.match;header=false,delim=","))
    append!(df,d)
end

In [38]:
typeof(regexp)

Regex

In [33]:
colnames=[:t,:o,:h,:l,:c,:v,:T,:q,:n,:V,:Q,:B]
tc1=[:t,:T]
tc2=[:opentime,:closetime]
@chain df begin
    rename!(colnames)
    transform!(tc1.=>(t->ts2dt.(t)).=>tc2)
end

,t,o,h,l,c,v,T,q,n,V,Q,B,opentime,closetime
,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Float64,Float64,Int64,DateTime,DateTime
1,1609459200000,736.42,737.09,735.94,737.02,647.72,1609459259999,4.76878e5,359,484.878,3.56959e5,0,2021-01-01T00:00:00,2021-01-01T00:00:59.999
2,1609459260000,737.12,739.0,737.06,738.74,1122.95,1609459319999,8.28662e5,643,801.453,5.91232e5,0,2021-01-01T00:01:00,2021-01-01T00:01:59.999
3,1609459320000,738.78,738.81,737.35,737.82,418.5,1609459379999,3.08923e5,283,202.663,1.49569e5,0,2021-01-01T00:02:00,2021-01-01T00:02:59.999
4,1609459380000,737.74,738.2,737.28,737.43,305.984,1609459439999,2.25753e5,251,235.368,1.73656e5,0,2021-01-01T00:03:00,2021-01-01T00:03:59.999
5,1609459440000,737.38,737.8,736.8,736.89,350.363,1609459499999,2.58306e5,277,217.562,1.60413e5,0,2021-01-01T00:04:00,2021-01-01T00:04:59.999
6,1609459500000,737.06,737.11,735.46,735.69,654.404,1609459559999,4.81658e5,494,263.41,1.93853e5,0,2021-01-01T00:05:00,2021-01-01T00:05:59.999
7,1609459560000,735.61,736.64,735.51,735.84,402.972,1609459619999,2.96731e5,333,243.017,1.7895e5,0,2021-01-01T00:06:00,2021-01-01T00:06:59.999
8,1609459620000,735.84,736.07,734.99,735.0,353.131,1609459679999,2.59689e5,359,218.661,160782.0,0,2021-01-01T00:07:00,2021-01-01T00:07:59.999
9,1609459680000,735.0,735.08,733.26,734.25,637.156,1609459739999,4.67713e5,604,292.028,2.1439e5,0,2021-01-01T00:08:00,2021-01-01T00:08:59.999


In [34]:
arrow_path = abspath("../data/arrow/ETHUSDT-1m-2021.arrow")
arrow_write(df,arrow_path)

In [35]:
df = arrow_read(arrow_path)
df

,t,o,h,l,c,v,T,q,n,V,Q,B,opentime,closetime
,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Float64,Float64,Int64,DateTime,DateTime
1,1609459200000,736.42,737.09,735.94,737.02,647.72,1609459259999,4.76878e5,359,484.878,3.56959e5,0,2021-01-01T00:00:00,2021-01-01T00:00:59.999
2,1609459260000,737.12,739.0,737.06,738.74,1122.95,1609459319999,8.28662e5,643,801.453,5.91232e5,0,2021-01-01T00:01:00,2021-01-01T00:01:59.999
3,1609459320000,738.78,738.81,737.35,737.82,418.5,1609459379999,3.08923e5,283,202.663,1.49569e5,0,2021-01-01T00:02:00,2021-01-01T00:02:59.999
4,1609459380000,737.74,738.2,737.28,737.43,305.984,1609459439999,2.25753e5,251,235.368,1.73656e5,0,2021-01-01T00:03:00,2021-01-01T00:03:59.999
5,1609459440000,737.38,737.8,736.8,736.89,350.363,1609459499999,2.58306e5,277,217.562,1.60413e5,0,2021-01-01T00:04:00,2021-01-01T00:04:59.999
6,1609459500000,737.06,737.11,735.46,735.69,654.404,1609459559999,4.81658e5,494,263.41,1.93853e5,0,2021-01-01T00:05:00,2021-01-01T00:05:59.999
7,1609459560000,735.61,736.64,735.51,735.84,402.972,1609459619999,2.96731e5,333,243.017,1.7895e5,0,2021-01-01T00:06:00,2021-01-01T00:06:59.999
8,1609459620000,735.84,736.07,734.99,735.0,353.131,1609459679999,2.59689e5,359,218.661,160782.0,0,2021-01-01T00:07:00,2021-01-01T00:07:59.999
9,1609459680000,735.0,735.08,733.26,734.25,637.156,1609459739999,4.67713e5,604,292.028,2.1439e5,0,2021-01-01T00:08:00,2021-01-01T00:08:59.999
